In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import techdata_to_tree, sys, ShockFunction
import abatement_ID_calib as abatement
os.chdir(directory['curr'])
def flatten_list(list_):
    return [item for sublist in list_ for item in sublist]

The file_gams_py_gdb0.gdx is still active and was not deleted.


## 1: Set up tree and read in data, akin to A1\_ID\_Calib

*Read data:*

In [2]:
inputfile = "techdata_dors_2.xlsx" # file
inputoth  = 'Othdata_dors_2.xlsx'
output = techdata_to_tree.load_techcats(pd.read_excel(directory['data'] + "/" + inputfile, sheet_name=["inputdisp", "endofpipe", "inputprices"]))
modules = ["ID", "EOP"] # modules to use

*Set up trees:*

In [3]:
nts = {m: nesting_tree.nesting_tree(name=m) for m in modules}

#### 1.1: Input displacing tree

In [4]:
nts["ID"].add_tree(output["ID"]["upper_categories"], tree_name = 'ID_EC', **{"type_f":"CES_norm"}) # E to C
nts["ID"].add_tree(output["ID"]["components"], tree_name = "ID_CU", **{"type_f":"MNL"}) # C to U
nts["ID"].add_tree(output["ID"]["techs"], tree_name="ID_TU", **{'type_io': 'output', 'type_f': 'CET_norm'}) # U to T
nts["ID"].add_tree(output["ID"]["techs_inputs"], tree_name="ID_TX") # T to inputs X
nts["ID"].add_tree(output["ID"]["basetechs"], tree_name="ID_BU", **{"type_io":"output", "type_f":"linear_out"}) # baseline technologies
nts["ID"].add_tree(output["ID"]["basetech_inputs"], tree_name="ID_BX")

#### 1.2: End of pipe tree

In [5]:
nts["EOP"].add_tree(output["EOP"]["components"], tree_name = "EOP_CU", **{"type_f":"MNL"}) # C to U
nts["EOP"].add_tree(output["EOP"]["techs"], tree_name="EOP_TU", **{'type_io': 'output', 'type_f': 'CET_norm'}) # T to U 
nts["EOP"].add_tree(output["EOP"]["techs_inputs"], tree_name="EOP_TX") # T to inputs X

#### 1.3: Trees related to final goods and emissions account

*Read in tree and data from xlsx:*

In [6]:
nts['ID'].add_tree(directory['data']+'\\'+inputoth,tree_name='ID_Y',**{'sheet':'Y'})
DataBase.GPM_database.merge_dbs(nts['ID'].trees['ID_Y'].database,excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth,{'Y': 'vars'}), 'first')

*Add new Q2P:*

In [7]:
output['ID']['Q2P'] = output['ID']['Q2P'].append(excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth,{'Q2P': 'maps'}).database['Q2P'].vals)

#### 1.3: Namespaces

*Give modules local namespaces:*

In [8]:
standard_sets = ('inp','out','int','wT','map_all','kno_out','kno_inp')
for module in modules:
    namespace = {k: module + '_' + k for k in standard_sets}
    nts[module].run_all(**namespace)
    #Also replaces keys with module-specific names, i.e. changes 'inp' to 'ID_inp' in the attributes/keys.
    for std_set in standard_sets:
        if hasattr(nts[module], std_set):
            setattr(nts[module], module + "_" + std_set, getattr(nts[module], std_set))
            delattr(nts[module], std_set)

*Add emissions data:*

In [9]:
DataBase.GPM_database.merge_dbs(nts['ID'].database,excel2py.xl2PM.pm_from_workbook(directory['data']+'\\'+inputoth, {'M': 'vars','M_sets': 'sets'}),'first')

## 2: Initialize model

*Initialize:*

In [12]:
output.keys()

dict_keys(['ID', 'PwT', 'EOP'])

In [10]:
gm = abatement.abate(nt=nts['ID'],tech=output,work_folder=directory['work'],**{'data_folder':directory['gams'],'name': 'atest'})

KeyError: 'PwT'

*Add a couple of extra aliases, and update sets with elements:*

In [ ]:
gm.add_aliases([('n','nnnn'),('n','nnnnn')])
gm.model.database.update_all_sets(clean_up=False)

In [ ]:
gm.initialize_variables_leontief()
gm.initialize_variables(**{'check_variables': True})

In [17]:
gm.get('currapp_ID')

n   nn         
EL  ID_t2          0.05
    basetech_EL    0.05
Name: currapp_ID, dtype: float64

## A: Additional equations

### Current application

* Define currapp\_ID[n,nn] as the share of energy-services covered by technology goods produced by technology $nn$ to satisfy the energy-service need $n$. In other words:

$$\begin{align}
    \text{currapp_ID[n,nn]} =\text{sum(nnn\$(e2t[n,nn] and e2u[n,nnn]), qD[nnn])} / qD[n].
\end{align}$$